In [11]:
# ライブラリのインポート
import numpy as np
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers

np.random.seed(123)
torch.manual_seed(123)

In [12]:
# データの生成
N = 300
x, t = datasets.make_moons(N, 0.3)
t = t[:, np.newaxis]
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.2)

In [13]:
# 多層パーセプトロン
class MLP(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim, output_dim)
        self.a1 = nn.Sigmoid()
        self.l2 = nn.Linear(hidden_dim, output_dim)
        self.a2 = nn.Sigmoid()
        self.layers = [self.l1, self.a1, self.l2, self.a2]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [25]:
# モデルの学習
# アルゴリズムの定義
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(2, 3, 1).to(device)
criterion = nn.BCELoss()
optimizer = optimizers.SGD(model.parameters(), lr=0.1)

def compute_loss(t, y):
    return criterion(y, t)

def train_step(x, t):
    model.train()
    preds = model(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

# アルゴリズムの設定
epochs = 100
batch_size = 10
n_batches = x_train.shape[0] // batch_size

In [30]:
# 確率的勾配法でモデルを学習
for epoch in range(epochs):
    train_loss = 0.0
    x_, t_ = shuffle(x_train, t_train)
    x_ = torch.Tensor(x_).to(device)
    t_ = torch.Tensor(t_).to(device)

    for n_batch in range(n_batches):
        start = n_batch * batch_size
        end = start + batch_size
        loss = train_step(x_[start:end], t_[start:end])
        train_loss += loss.item()

    print("epoch: {}, loss: {:.3}".format(epoch+1, train_loss))

epoch: 1, loss: 16.5
epoch: 2, loss: 16.0
epoch: 3, loss: 15.6
epoch: 4, loss: 15.2
epoch: 5, loss: 14.7
epoch: 6, loss: 14.2
epoch: 7, loss: 13.6
epoch: 8, loss: 13.1
epoch: 9, loss: 12.6
epoch: 10, loss: 12.1
epoch: 11, loss: 11.6
epoch: 12, loss: 11.2
epoch: 13, loss: 10.8
epoch: 14, loss: 10.4
epoch: 15, loss: 10.1
epoch: 16, loss: 9.81
epoch: 17, loss: 9.54
epoch: 18, loss: 9.29
epoch: 19, loss: 9.08
epoch: 20, loss: 8.87
epoch: 21, loss: 8.67
epoch: 22, loss: 8.51
epoch: 23, loss: 8.33
epoch: 24, loss: 8.17
epoch: 25, loss: 8.05
epoch: 26, loss: 7.92
epoch: 27, loss: 7.8
epoch: 28, loss: 7.68
epoch: 29, loss: 7.56
epoch: 30, loss: 7.47
epoch: 31, loss: 7.37
epoch: 32, loss: 7.28
epoch: 33, loss: 7.19
epoch: 34, loss: 7.12
epoch: 35, loss: 7.05
epoch: 36, loss: 6.98
epoch: 37, loss: 6.93
epoch: 38, loss: 6.86
epoch: 39, loss: 6.81
epoch: 40, loss: 6.77
epoch: 41, loss: 6.73
epoch: 42, loss: 6.68
epoch: 43, loss: 6.65
epoch: 44, loss: 6.61
epoch: 45, loss: 6.57
epoch: 46, loss: 6.5

In [45]:
# モデルの評価
def test_step(x, t):
    x = torch.Tensor(x).to(device)
    t = torch.Tensor(t).to(device)
    model.eval()
    preds = model(x)
    loss = compute_loss(t, preds)
    return loss, preds

In [50]:
loss, preds = test_step(x_test, t_test)
test_loss = loss.item()
preds = preds.data.cpu().numpy() > 0.5
test_acc = accuracy_score(t_test, preds)
print(test_loss, test_acc)

0.2655848562717438 0.9
